In [ ]:
import httplib2
import oauth2
import urllib3
import types
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gender_detector import GenderDetector 
import psycopg2, psycopg2.extras
from causalinference import CausalModel
from causalinference.utils import random_data
import httplib
import base64
import json # For Microsoft Face API
import urllib as urllib # For Microsoft Face API
import time 
import csv
import datetime 


class UpworkDataFormatter:
    
    def __init__(self):
        # Settings
        self.edu_data_file_name = './csv_files/edu_groundtruth_2017_12_12_upwork_analysis_unitedstates_allskills.csv' # Filename for all data
        self.edu_data_log_file_name = './log_files/edu_groundtruth_log_upwork_data_analysis_2017_12_12_unitedstates_allskills.txt'
        self.ground_truth_file_name = './csv_files/interrater_2017_12_12_upwork_analysis_unitedstates_allskills.csv'
        
        # Write a log
        self.log = open(self.edu_data_log_file_name, 'a')
        self.log.write("We have started analyzing data!" + "\n")
        self.log.flush()

        # Connect to the database 
        self.conn = psycopg2.connect("dbname=eureka01")
        self.cur = self.conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
        psycopg2.extensions.register_adapter(dict, psycopg2.extras.Json)
        
        # Initialize values 
        self.user_count = 1
        self.education_levels = ["Bachelor or Higher", "Some College or Associate",
                                 "High School", "Less Than High School", "Other", "None"]
        self.worker_id_list = self.get_ids_from_dataset(self.ground_truth_file_name)
    
    def show_education(self):

        with open(self.edu_data_file_name, 'w') as csvfile:
            fieldnames = ['user_count','education','education_id']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            
            for worker in self.worker_id_list:
                self.cur.execute("""SELECT detailed_info FROM upwork_unitedstates_allskills_2017_12_12 WHERE user_id = %s;""", (worker,))
                
                for user in self.cur:
                    try: 
                        print "This is the user count: {0}".format(self.user_count)
                        user_count = self.user_count
                        education_list = self.return_edu_experience(user)
                        highest_education = self.find_highest_education(education_list)
                        education_id = str(self.education_levels.index(highest_education))

                        print "----This is the highest education list {0}----".format(education_list)
                        print "------This is highest education: {0}, {1}-----".format(highest_education, type(highest_education))
                        print "------This is education id: {0}, {1}------".format(education_id, type(education_id))

                        writer.writerow({'user_count': self.user_count, 'education': highest_education, 
                                     'education_id': education_id})
                        print "Finished writing data for {0}".format(self.user_count)

                    except:
                        print "Ran into some error at user {0}".format(self.user_count)

                        writer.writerow({'user_count': self.user_count, 'education': "error", 
                                     'education_id': "error"})

                self.user_count += 1
                
                    
    def find_highest_education(self, edu_list):
        for level in self.education_levels:
            if level in edu_list:
                return level
            else:
                return "None"
                
    def return_edu_experience(self, user):
        all_education = []
        try: 
            ed_history = user[0]["education"]["institution"]
            
            if (ed_history == ""):
                print "No ed history"
                return all_education
            
            if (type(ed_history) == dict):
                print "Found a single ed history"
                degree = str(ed_history["ed_degree"])
                school = str(ed_history["ed_school"])
                comment = str(ed_history["ed_comment"])
                
                print "Degree: {0}".format(degree)
                print "School: {0}".format(school)
                print "Comment: {0}".format(comment)
                
                education = self.check_education(degree, school, comment)
                
                has_bachelor_or_higher = re.search("Professional|Doctorate|Master|Bachelor", education)
                has_some_college = re.search("Associate|Certificate|Some College", education)
                has_high_school = re.match("High School", education)
                has_less_than_high_school = re.match("Less Than High School", education)
                
                if (has_high_school): # Check for high school first 
                    all_education.append("High School")
                elif (has_less_than_high_school):
                    all_education.append("Less Than High School")
                elif (has_bachelor_or_higher):
                    all_education.append("Bachelor or Higher")
                elif (has_some_college):
                    all_education.append("Some College or Associate")
                else:
                    all_education.append("Other")
                
                return all_education

            elif (type(ed_history) == list):
                for item in ed_history:
                    degree = str(item["ed_degree"])
                    school = str(item["ed_school"])
                    comment = str(item["ed_comment"])
                    
                    print "Degree: {0}".format(degree)
                    print "School: {0}".format(school)
                    print "Comment: {0}".format(comment)
                
                    education = self.check_education(degree, school, comment)
                    has_bachelor_or_higher = re.search("Professional|Doctorate|Master|Bachelor", education)
                    has_some_college = re.search("Associate|Certificate|Some College", education)
                    has_less_than_high_school = re.search("Less Than High School", education)
                    has_high_school = re.search("High School", education)
                    
                    if (has_high_school): # Check for high school first 
                        all_education.append("High School")
                    elif (has_less_than_high_school):
                        all_education.append("Less Than High School")
                    elif (has_bachelor_or_higher):
                        all_education.append("Bachelor or Higher")
                    elif (has_some_college):
                        all_education.append("Some College or Associate")
                    else:
                        all_education.append("Other")
                        
                return all_education
        
        except Exception as error:
            print "We ran into an error at return edu experience"
            print error
            return all_education
        
    def check_education(self, degree, school, comment): # Do a quick check that the user has a high school degree
        try:
            hs_degree_mentioned = False
            diploma_degree_mentioned = False
            hs_word_mentioned = False
            graduated_mentioned = False
            
            if degree != "":
                hs_degree_mentioned = re.search("high school", degree.lower())
                diploma_degree_mentioned = re.search("diploma", degree.lower())
            
            if school != "":
                hs_word_mentioned = re.search("high school", school.lower())
            
            if comment != "":
                graduated_mentioned = re.search("graduated", comment.lower())

            if (hs_degree_mentioned or (hs_word_mentioned and (graduated_mentioned or diploma_degree_mentioned))):
                return "High School"
            elif (hs_word_mentioned and degree == ""):
                return "Less Than High School"
            elif (self.check_school_by_name("high_school", school)): # Check if the school name is in the database
                print "We found the high school!"
                if (hs_degree_mentioned or diploma_degree_mentioned or graduated_mentioned):
                    return "High School"
                elif (degree == ""):
                    return "Less Than High School"
            else:
                print "We did not find the high school; We should be checking the specific degree now!! "
                return self.check_specific_degree(degree, school, comment)
        except Exception as error:
            print "Something went wrong"
            print error
            return "Error"
        
    def check_specific_degree(self, degree, school, comment): # Do a quick check that the user has a bachelor or associate degree

        try:
            
            # If there is no degree, then check if they are enrolled in a college in the US
            if (degree == ""):
                print "Person has no degree; checking if they are in a college in the US"
                if (self.check_school_by_name("college", school)):
                    return "Some College"
                else:
                    return "Ambiguous"
            
            # Professional degree
            if (re.search('d\.c|d\.c\.m|d\.d\.s|d\.m\.d|ll\.b|ll\.m|l\.l\.m|l\.l\.b|j\.d\.|m\.d\.|o\.d\.|d\.o\.|pharm\.d', degree.lower()) 
                or re.search('d\.p\.m|d\.p|pod\.d|m\.div|m\.h\.l|b\.d|ordination|d\.v\.m| law', degree.lower())
                or re.search('DC|DCM|DDS|DMD|LLB|LLM|LLB|JD|MD|OD|DO|PharmD|Ed\.D|EdD|CFA', degree)
                or re.search('DPM|DP|PodD|MDiv|MHL|BD|DVM', degree)):
                return "Professional"

            # PhD
            elif (re.search('Ph\.D|PhD', degree)
                 or re.search('doctor|licentiate', degree.lower())): # Doctorate placed here in case MDs are also called doctors 
                return "Doctorate"

            # Masters
            elif (re.search('M\.B\.A|MBA|M\.A|M\.S|M\.Sc|M\.E|MA|MS|MSc|ME|M\.A\.Ed|MAEd|MST|M\.S\.T|MEd|M\.Ed|MPA|M\.P\.A', degree)
                 or re.search('master|engineer\'s degree', degree.lower())
                 or re.search('MFA|M\.F\.A|MSC|MCS', degree)):
                return "Master"

            # Bachelors
            elif (re.search('B\.A|B\.S|B\.Sc|BA|BS|BSc|BFA|AB|A\.B|BPharm|B\.Pharm', degree)
                 or re.search('bachelor|bpharm|bachlor|batchelor', degree.lower())):
                return "Bachelor"

            # Associate's Degree
            elif (re.search('AA|AS|A\.A|A\.S|A\.A\.S|AAS', degree)
                 or re.search('associate|foundation', degree.lower())):
                return "Associate"

            # Certificate
            elif (re.search('certificate', degree.lower())):
                return "Certificate"
            
            else:
                print "Ambiguous" 
                return "Ambiguous"
                
            
        except Exception as error:
            print "We ran into an error specifying the degree"
            print error
            return "Error"

            
    def check_school_by_name(self, dataset, name): # Checks if school name is in a dataset of schools 
        name = name.lower()
        name = self.remove_punctuation(name)
        private_schools = 'high_schools_in_us.csv'
        public_schools = 'high_schools_in_us.csv'
        colleges = 'colleges_in_us.csv'
        
        if (dataset == "high_school"): # Check for "high school" and against list of private and public high schools
            if re.search('highschool', name):
                return True
            
            with open(private_schools, 'r') as f:
                private_school_data = csv.DictReader(f)
                for row in private_school_data:
                    school = row["pinst"].lower()
                    school = self.remove_punctuation(school)
                    if (name == school):
                        return True
                    
            with open(public_schools, 'r') as f: # Haven't been able to find a suitable dataset 
                public_school_data = csv.DictReader(f)
                for row in public_school_data:
                    school = row["pinst"].lower()
                    school = self.remove_punctuation(school)
                    if (name == school):
                        return True
            
            return False # Return false if we can't find the school name in any of the lists
                
        elif (dataset == "college"): # Check for keywords and against list of colleges
            if (re.search('college|university|institute', name)):
                return True 
            with open(colleges, 'r') as f:
                college_data = csv.DictReader(f)
                for row in college_data:
                    school = row["INSTNM"].lower()
                    school = self.remove_punctuation(school)
                    if (name == school):
                        print "Found a matching college!"
                        print name
                        return True
            
            return False # Return false if we can't find the school name in any of the lists
        
        else:
            return False # Return false if all else fails 
    
    def remove_punctuation(self, word): # Removes all punctuation and blank spaces from a string
        modified_word = re.sub(r'[^\w\s]','',word)
        modified_word = re.sub(' ', '', modified_word)
        return modified_word
                    
    def show_degree(self, user):
        try: 
            ed_history = user[0]["education"]["institution"]
            if(ed_history == ""):
                return "No edu listed"
            elif(type(ed_history) == dict):
                return ed_history["ed_degree"]
            elif(type(ed_history) == list):
                degree_list = []
                for ed_experience in user[0]["education"]["institution"]:
                    if (ed_experience["ed_degree"] == ""):
                        degree_list.append("none")
                    elif (ed_experience["ed_degree"] != ""):
                        degree_list.append(ed_experience["ed_degree"])
                return degree_list
            else:
                return "Error: Some other error"
                
        except:
            return "Error: No education"
    
    def get_ids_from_dataset(self, spreadsheet):
        id_list = []
        df = pd.read_csv(spreadsheet)
        for row in df['worker_id']:
            print row
            id_list.append(row)
        return id_list
                    
myObject = UpworkDataFormatter()

myObject.show_education()

~010bd468bfc0423fd2
~01d435bf3e526173d6
~01caece2f244e9af27
~01ee2a8c8890151c1f
~01b7ba3e7659f8fa15
~01c4528d8604af2915
~019ace575609b0cfa8
~0185c4514e241b889c
~018b09bfb8b1ff46d8
~01a5bc6d2f8a824f67
~0161cb6b6f7775debd
~0148c79ceca7c9e77e
~013352bffe05d187cf
~011e3d50e821b39b77
~01e82981266a111ce6
~012c51352c78c1b441
~01f464d89fa067aec7
~01c98c8488669e961b
~01b98afa74c41a2292
~01fd28834f98d25381
~010821a5e7df72b57d
~01ffc8bf05fb309b5c
~012d44053709e3814f
~01ca5d03ca35d74bc5
~01f1195835f0077a7d
~01f2f34d2a4cf94b78
~01dfec01ff12e010c9
~0136ebaceb01f3991c
~01188050f758172b14
~01e2f01b219e0008f4
~01cad6188e12352724
~01ec5d19ce5c18bbe1
~014226e45a3d6482e1
~010a31dcb97f9e9df4
~01953cec30a7626914
~017616cc029be0ac9f
~011208a8367d29aa27
~0165a72fc6f65b0f0f
~010e12df92ef33e697
~01340a2f860dc75943
~019962ee3660e635f4
~01bf6d7e00a8a874b0
~01d7f9e241bb79ff02
~01f9dd565ab2480e52
~018349a31a94bc4246
~01fc1b7bfc3ea49959
~01350245e6bbbcf1a6
~01ebb44d6ddfd1dc36
~011fe37e39d22ccfd6
~017f2a7ea2084e2be7


Degree: 
School: Delaware Technical and Community College
Comment: 
We did not find the high school; We should be checking the specific degree now!! 
Person has no degree; checking if they are in a college in the US
----This is the highest education list ['Other', 'Some College or Associate']----
------This is highest education: None, <type 'str'>-----
------This is education id: 5, <type 'str'>------
Finished writing data for 8
This is the user count: 9
Found a single ed history
Degree: Bachelor of Science (B.S.)
School: San Diego State University
Comment: 
We did not find the high school; We should be checking the specific degree now!! 
----This is the highest education list ['Bachelor or Higher']----
------This is highest education: Bachelor or Higher, <type 'str'>-----
------This is education id: 0, <type 'str'>------
Finished writing data for 9
This is the user count: 10
Found a single ed history
Degree: Bachelor of Medicine, Bachelor of Surgery (M.B.B.S.)
School: Rajiv Gandhi Uni

We did not find the high school; We should be checking the specific degree now!! 
Degree: Bachelor of Arts (B.A.)
School: Ithaca College
Comment: Double minor in Writing and Politics
We did not find the high school; We should be checking the specific degree now!! 
----This is the highest education list ['Bachelor or Higher', 'Bachelor or Higher']----
------This is highest education: Bachelor or Higher, <type 'str'>-----
------This is education id: 0, <type 'str'>------
Finished writing data for 25
This is the user count: 26
Found a single ed history
Degree: Associate's degree
School: Tulsa Community College
Comment: 
We did not find the high school; We should be checking the specific degree now!! 
----This is the highest education list ['Some College or Associate']----
------This is highest education: None, <type 'str'>-----
------This is education id: 5, <type 'str'>------
Finished writing data for 26
This is the user count: 27
Found a single ed history
Degree: Bachelor of Business Ad

We did not find the high school; We should be checking the specific degree now!! 
----This is the highest education list ['Some College or Associate']----
------This is highest education: None, <type 'str'>-----
------This is education id: 5, <type 'str'>------
Finished writing data for 39
This is the user count: 40
Degree: Master of Arts (M.A.)
School: Texas A&M University
Comment: Critical Theory Analysis, Rhetorical Analysis, Quantitative Analysis, Political Rhetoric Analysis, Team Analysis, Organizational Communication, 
We did not find the high school; We should be checking the specific degree now!! 
Degree: Bachelor of Arts (B.A.)
School: McMurry University
Comment: Public Speaking, Oral Interpretation, Gender Communication, Mass Communication, Rhetoric, Persuasion, Group Communication, Interpersonal Communication.
Minor in Psychology
We did not find the high school; We should be checking the specific degree now!! 
----This is the highest education list ['Bachelor or Higher', 'Ba

We did not find the high school; We should be checking the specific degree now!! 
----This is the highest education list ['Some College or Associate', 'Bachelor or Higher']----
------This is highest education: Bachelor or Higher, <type 'str'>-----
------This is education id: 0, <type 'str'>------
Finished writing data for 52
This is the user count: 53
Found a single ed history
Degree: 
School: Ball State University
Comment: 
We did not find the high school; We should be checking the specific degree now!! 
Person has no degree; checking if they are in a college in the US
----This is the highest education list ['Some College or Associate']----
------This is highest education: None, <type 'str'>-----
------This is education id: 5, <type 'str'>------
Finished writing data for 53
This is the user count: 54
Found a single ed history
Degree: Bachelor of Engineering (B.Eng.)
School: Rungta Engineering College
Comment: 
We did not find the high school; We should be checking the specific degree 